In [1]:
suppressMessages(library(rwwa))

In [2]:
nsamp = 1000

varnm = "rx3day"

rp <- list("rx3day" = 10, "pr-jjaso" = 10)[[varnm]]
mdl_args <- list("rx3day" = list("dist" = "gev", "type" = "fixeddisp"),
                 "pr-jjaso" = list("dist" = "norm", "type" = "fixeddisp"))[[varnm]]

In [3]:
# standardise observed Nino to have mean 0 and sd 1 over 1979-2023
nino_obs <- load_ts("ts/mexico-floods_nino3.4-sep.dat", col.names = c("year", "nino"))
nino_obs <- rbind(nino_obs, c(2025, 26.0))

nino_1979 <- nino_obs$nino[nino_obs$year >= 1979]
nino_std <- data.frame("year" = nino_obs$year, "nino" = (nino_obs$nino - mean(nino_1979)) / sd(nino_1979))

# use 2025 standardised Nino as covariate
nino_2025 <- nino_std$nino[nino_std$year == 2025]

## CORDEX - trend with ENSO

In [6]:
fl <- list.files("ts", pattern = paste0(varnm, "_CAM"), full.names = T)

for (fnm in fl) {
    print(fnm)

    res_fnm <- gsub(".nc", ".csv", gsub("ts", "res-nino", fnm))
    if (exists(res_fnm)) next

    gcm <- paste0(strsplit(fnm, "_")[[1]][3:4], collapse = "_")
    
    nino_fnm <- list.files("ts", paste0("nino3.4-sep_", gcm), full.names = T)
    if (length(nino_fnm) == 0) next

    gmst_fnm <- list.files("ts", gsub("_", "_rcp85_", paste0(gsub("i1p1", "", gcm), ".dat")), full.names = T)
    if (length(gmst_fnm) == 0) next

    nino <- load_ts(nino_fnm, col.names = c("year", "nino34"))
    gmst <- load_ts(gmst_fnm, col.names = c("year", "gmst"))
    ts <- load_ts(fnm, col.names = c("year", "pr"))
    df <- data.frame(apply(merge(merge(nino, gmst), ts), 2, as.numeric))

    # add standardised Nino covariate
    nino_clim <- df[(df$year >= 1979) & (df$year <= 2025), c("nino34", "pr")]
    df$nino_std <- (df$nino - mean(nino_clim$nino34)) / sd(nino_clim$nino34)

    # trend fitting
    mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "pr", "covnm" = c("gmst", "nino_std"), "lower" = F)))

    # use model's 2023 GMST & observed Nino to define factual climate
    gmst_2025 <- df$gmst[df$year == 2025]
    
    cov_2025 <- data.frame(gmst = gmst_2025, nino_std = nino_2025)
    cov_cf <- rbind("pi" = data.frame("gmst" = gmst_2025 - 1.3, "nino_std" = nino_2025), 
                    "neut" = data.frame("gmst" = gmst_2025, "nino_std" = 0))
    cov_fut <- data.frame(gmst = gmst_2025 + 1.3, nino_std = nino_2025)

    # bootstrap results
    res <- cmodel_results(mdl, rp = rp, cov_f = cov_2025, cov_hist = cov_cf, cov_fut = cov_fut,
                         y_now = 2025, y_start = 1979, y_fut = 2070, nsamp = nsamp)

    # bootstrap Nino amplitude & correlation between Nino & precip in this region
    set.seed(42)

    nino_boot <- sapply(1:nsamp, function(i) {
        boot_df <- nino_clim[sample(1:nrow(nino_clim), replace = T), ]
        c("nino_cor" = cor(boot_df$nino34, boot_df$pr), "nino_sd" = sd(boot_df$nino34))
    })
    nino_qq <- rbind("est" = c(cor(df$nino34, df$pr), sd(df$nino34)), apply(nino_boot,1,quantile,c(0.025, 0.975)))
    nino_qq <- t(data.frame(unlist(sapply(colnames(nino_qq), function(cnm) {
        setNames(nino_qq[,cnm], paste0(cnm,"_", c("est", "lower", "upper")))
    }, simplify = F, USE.NAMES = F))))
                                       
    res <- cbind(res, nino_qq)
    write.csv(res, res_fnm)

    # quick return level plots to check fit
    rlplot_fnm <- gsub("csv", "png", gsub("res-nino/", "mdl-eval/rlplot-nino_", res_fnm))
    if(!file.exists(rlplot_fnm)) {
        png(rlplot_fnm, h = 480, w = 480 * 2); par(cex.main = 1.4, mfrow = c(1,2), oma = c(0,0,2,0)); {
            plot_returnlevels(mdl, cov_f = cov_2025, cov_cf = cov_cf["pi",,drop = F], nsamp = 100, main = "Present vs PI")
            plot_returnlevels(mdl, cov_f = cov_2025, cov_cf = cov_cf["neut",,drop = F], nsamp = 100, main ="Present vs neutral Nino3.4")
            mtext(outer = T, gsub(".dat","",paste0(strsplit(fnm, "_")[[1]][3:5], collapse = "_")), font = 2, cex = 1.5)
        }; dev.off()
    }
}

[1] "ts/pr-rx3day_CAM-22_CanESM2_r1i1p1_CRCM5_19500101-20701231.nc"
[1] "ts/pr-rx3day_CAM-22_GFDL-ESM2M_r1i1p1_CRCM5_19500101-20701231.nc"
[1] "ts/pr-rx3day_CAM-22_GFDL-ESM2M_r1i1p1_RegCM4-7_19700101-20691231.nc"
[1] "ts/pr-rx3day_CAM-22_HadGEM2-ES_r1i1p1_RegCM4-7_19700101-20691230.nc"
[1] "ts/pr-rx3day_CAM-22_HadGEM2-ES_r1i1p1_REMO2015_19700101-20701230.nc"
[1] "ts/pr-rx3day_CAM-22_MPI-ESM-LR_r1i1p1_REMO2015_19700101-20701231.nc"
[1] "ts/pr-rx3day_CAM-22_MPI-ESM-MR_r1i1p1_RegCM4-7_19700101-20691231.nc"
